In [25]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [2]:
pagos_efect = "BASE_PAGOS_EFECTIVIDADES.xlsx"
df_pagos_efect = pd.read_excel(pagos_efect, sheet_name="BD")

In [ ]:
df_pagos_test = df_pagos_efect.copy()
print(df_pagos_test.shape)
print(df_pagos_test.dtypes)
df_pagos_test.head(3)

In [ ]:
df_pagos_test.columns

In [ ]:
cols =  ['Agencia', 'clave', 'SEGMENTO_RIESGO', 'AMBITO_RCD_FINAL', 'CapitalSoles']
df_pagos_test = df_pagos_test[cols]
print(df_pagos_test.shape)

In [ ]:
df_pagos_test.rename(columns={'clave': 'CLAVE', 'Agencia': 'AGENCIA', 'CapitalSoles': 'CAPITAL', 'AMBITO_RCD_FINAL': 'PRODUCTO', 'SEGMENTO_RIESGO': 'SEGMENTO'}, inplace=True)
df_pagos_test.head()

In [ ]:
df_pagos_test = df_pagos_test[df_pagos_test['AGENCIA'].isin(["ASESCOM RJ", "CLASA MORA", "MORNESE MORA", "SIN AGENCIA"])]
df_pagos_test["AGENCIA"].unique()

In [ ]:
df_pagos_test["AGENCIA"].value_counts()

In [ ]:
df_pagos_test['SEGMENTO'] = df_pagos_test['SEGMENTO'].apply(lambda x: 'PYME / EMP MIN' if x != 'PARTICULARES' else x)
df_pagos_test['SEGMENTO'].unique()

In [ ]:
df_pagos_test["SEGMENTO"].value_counts()

In [ ]:
df_pagos_test['PRODUCTO'] = df_pagos_test['PRODUCTO'].str.replace(r'\d+', '', regex=True).str.replace('.', '').str.upper()
df_pagos_test['PRODUCTO'] = df_pagos_test['PRODUCTO'].str.strip().replace(' ', '')
df_pagos_test['PRODUCTO'].unique()

In [ ]:
df_pagos_test["PRODUCTO"].value_counts()

In [ ]:
df_pagos_test['CAPITAL'] = df_pagos_test['CAPITAL'].round(2)
df_pagos_test.head()

In [ ]:
df_final = df_pagos_test.groupby(['AGENCIA', 'CLAVE', 'SEGMENTO', 'PRODUCTO']).agg({'CAPITAL': 'sum'}).reset_index()
df_final.head()

In [24]:
df_final.to_excel("BASE_PAGOS_EFECTIVIDADES_SEPTIEMBRE.xlsx", index=False)